# Давай потыкаемся в реализацию

In [1]:
import sys 
sys.path.append("..")

import torch
from models.quantization_utils.quant_utils import fixedpoint_mul
from quant_train import deit_tiny_patch16_224



In [2]:
nb_classes = 1000
drop = 0.
drop_path = 0.1

model = deit_tiny_patch16_224(
    pretrained=True,
    num_classes=nb_classes,
    drop_rate=drop,
    drop_path_rate=drop_path
)
model.eval()

VisionTransformer(
  (qact_input): QuantAct(activation_bit=8, quant_mode: symmetric
  (patch_embed): PatchEmbed(
    (proj): (QuantConv2d(3, 192, kernel_size=(16, 16), stride=(16, 16)) weight_bit=8, quant_mode=symmetric)
    (qact): QuantAct(activation_bit=16, quant_mode: symmetric
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (qact_pos): QuantAct(activation_bit=16, quant_mode: symmetric
  (qact1): QuantAct(activation_bit=16, quant_mode: symmetric
  (blocks): ModuleList(
    (0): Block(
      (norm1): IntLayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (qact1): QuantAct(activation_bit=8, quant_mode: symmetric
      (attn): Attention(
        (qkv): (QuantLinear(in_features=192, out_features=576, bias=True) weight_bit=8, quant_mode=symmetric)
        (qact1): QuantAct(activation_bit=8, quant_mode: symmetric
        (qact_attn1): QuantAct(activation_bit=8, quant_mode: symmetric
        (qact2): QuantAct(activation_bit=8, quant_mode: symmetric
        (proj): (QuantLinear(i

In [3]:


B = 1
T = 128
D = 32

x = torch.randint(-127, 127, size=(B, T, D))
x

tensor([[[  -1,   42, -127,  ...,   35,  122,  -31],
         [  86,  121,   62,  ...,  -87,   94,   62],
         [  27,  -62,  -45,  ..., -101, -118,  -38],
         ...,
         [  51,    1,  -29,  ...,  -60, -119,   18],
         [   5,   23,   77,  ...,  100,   90,   61],
         [-119, -107,  -46,  ...,  -77,   95,   28]]])

In [4]:
pre_act_scaling_factor = torch.tensor(1 / 127)
activation_bit = 9
quant_mode = 'symmetric'
act_scaling_factor = torch.tensor(1 / 64)

In [5]:
quant_act_int = fixedpoint_mul.apply(
    x, pre_act_scaling_factor,
    activation_bit, quant_mode,
    act_scaling_factor,
)

In [6]:
quant_act_int

tensor([[[  -64.,  2688., -8128.,  ...,  2240.,  7808., -1984.],
         [ 5504.,  7744.,  3968.,  ..., -5568.,  6016.,  3968.],
         [ 1728., -3968., -2880.,  ..., -6464., -7552., -2432.],
         ...,
         [ 3264.,    64., -1856.,  ..., -3840., -7616.,  1152.],
         [  320.,  1472.,  4928.,  ...,  6400.,  5760.,  3904.],
         [-7616., -6848., -2944.,  ..., -4928.,  6080.,  1792.]]])